Author: Ronny F. Pacheco Date: Nov 2024 Copyright: © 2024 Ronny Pacheco License: MIT License

MIT License

Copyright (c) 2024 Ronny Pacheco

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


# In/Out files

- In files:
- Out files:

# Needed modules

In [25]:
import pandas as pd
import pickle
import os
import subprocess

from jupyter_core.version import pattern

In [26]:
# https://kioku-space.com/en/jupyter-skip-execution/
from IPython.core.magic import register_cell_magic # type: ignore


@register_cell_magic
def skip(line, cell):
    print('Skipping cell')
    if line and cell:
        pass
    return

# Pickle save

In [27]:
%%skip
# =============================================================================
# main function
# =============================================================================
def data_save_load(option):
    """
    This function is used to save or load data for the jupyter notebook
    """
    path_folder = "ipynb_db"  # Folder to save variables
    os.makedirs(path_folder, exist_ok=True)  # Create folder if not exist
    notebook_name = '1.prepare_data'
    path_file = os.path.join(path_folder, f"{notebook_name}.variables.pkl") # Path to save the variables

    if option == "save":
        with open(path_file, "wb") as pickle_file:
            dict_variables = { # 'var_name' : 'real_var'

            }
            pickle.dump(dict_variables, pickle_file)
    elif option == "load":
        with open(path_file, "rb") as pickle_file:
            variables = pickle.load(pickle_file)
        # Now load the variables
        for pickle_key, pickle_value in variables.items():
            print(f"* Loading variable: {pickle_key}")
            globals()[pickle_key] = pickle_value
# =============================================================================
# Call the function
# =============================================================================
data_save_load(option="load")

Skipping cell


# Load data

## SIDER form our software

In [28]:
# Load sider merged data
sider_df = pd.read_csv('./data/sider_merged.csv', sep=',', header=0)
print(sider_df.shape)
print(sider_df.dtypes)
sider_df.head()

(2874, 5)
chrom          object
sider_start     int64
sider_end       int64
strand         object
sider_seq      object
dtype: object


,chrom,sider_start,sider_end,strand,sider_seq
0,LmjF.01,24284,24536,plus,GCAGCCCCCTCCCCCTCTATCCCTGCCAACGCCAAACCACTTCTGG...
1,LmjF.01,24604,25200,plus,CCGGCATGATGGAGGGGCGGCTGTGAGGTGGTGCGTGCATGTATGT...
2,LmjF.01,35467,36401,plus,CCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACACGCGCTGCAG...
3,LmjF.01,39730,40541,plus,ATTTTACCGCGAGCAAGGCAACACACAGACGCACACACAACCACAG...
4,LmjF.01,54594,55542,plus,GCCGCACCCCACCCCCATCGCCTGGTGCGAAGCAGCGCAAGACACA...


## Bringaud SIDER

In [29]:
# Load bringaud sider data
bringaud_df = pd.read_csv('../data/bringaud_siders.csv', sep=',', header=0)
print(bringaud_df.shape)
print(bringaud_df.dtypes)
bringaud_df.head()

(1852, 8)
ID          object
chr          int64
name        object
fam          int64
str         object
start        int64
end          int64
chr_size     int64
dtype: object


,ID,chr,name,fam,str,start,end,chr_size
0,51.te00022,1,chr1_37.00281,1,-,39705,40532,268984
1,51.te00019,1,chr1_37.00289,1,+,260993,261261,268984
2,51.te00020,1,chr1_37.00290,1,+,266162,266340,268984
3,51.te00018,1,chr1_37.00279,2,+,24174,24821,268984
4,51.te00021,1,chr1_37.00280,2,-,35328,36036,268984


# Prepare data

## Bringaud siders

In [30]:
# Take from bringaud only the needed data
bringaud_df = bringaud_df[['name', 'start', 'end']]
bringaud_df = bringaud_df.rename(
    columns={'start' : 'bringaud_start', 'end' : 'bringaud_end'}
)
bringaud_df.head()

,name,bringaud_start,bringaud_end
0,chr1_37.00281,39705,40532
1,chr1_37.00289,260993,261261
2,chr1_37.00290,266162,266340
3,chr1_37.00279,24174,24821
4,chr1_37.00280,35328,36036


In [31]:
# Get the chromosome and prepend '0' to single digits, then add 'LmJF.'
pattern = r'^\D+(\d+)_'
bringaud_df['name'] = bringaud_df['name'].str.extract(pattern, expand=True)

# Now for the numbers like 0, 1, 2.. get 01, 02, 03...
bringaud_df['name'] = 'LmJF.' + bringaud_df['name'].str.zfill(2)

bringaud_df.head()

,name,bringaud_start,bringaud_end
0,LmJF.01,39705,40532
1,LmJF.01,260993,261261
2,LmJF.01,266162,266340
3,LmJF.01,24174,24821
4,LmJF.01,35328,36036


In [33]:
# Check if 'bringaud_start' > 'bringaud_end'. It should be 0
bringaud_df[bringaud_df['bringaud_start'] > bringaud_df['bringaud_end']].shape

(0, 3)